# Complex Mosaicing


Optimizes the image coverage search to avoid scene overlaps. Only downloads the neccessary parts.

Download the overlap area, and each unique area. Each in one job. For the overlap area the scene with the lowest cloudcover.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import copy

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.plot import show

import up42

from utils.geo import buffer_meter, add_area_in_sqkm, explode_mp, get_best_sections_full_coverage

## Define aoi

In [ ]:
# Catalog currently only works with account authentification!
api = up42.authenticate(cfg_file="config.json", env="dev")

In [ ]:
outdir = Path.cwd() / "dakar_complex_pleiades_fullscene"
outdir.mkdir(parents=True, exist_ok=True)

In [ ]:
aoi = up42.read_vector_file("aois/dakar.geojson", as_dataframe=True)
display(aoi)
aoi.plot()

## Search available images

In [ ]:
catalog = up42.initialize_catalog()
search_paramaters = catalog.construct_parameters(geometry=aoi, 
                                                 start_date="2018-08-02",
                                                 end_date="2020-08-09",
                                                 sensors=["pleiades"],
                                                 max_cloudcover=20,
                                                 sortby="cloudCoverage", 
                                                 limit=10)
#search_paramaters

In [ ]:
# Search & Visualize
search_results = catalog.search(search_paramaters=search_paramaters, as_dataframe=True)
search_results["incidenceAngle"] = search_results["providerProperties"].apply(lambda x : x["incidenceAngle"])
display(search_results)
catalog.plot_coverage(scenes=search_results, aoi=aoi)

# Reduce columns & export
df = search_results[["geometry", "id", "scene_id", "cloudCoverage", "blockNames", "incidenceAngle"]]
df["blockNames"] = df["blockNames"].to_string()
df.to_file(driver="GeoJSON", filename=outdir / "search_results_limited_columns.geojson")

In [ ]:
# Quicklooks
#catalog.download_quicklooks(image_ids=search_results["id"].tolist(), provider="oneatlas", output_directory=outdir / "quicklooks")
#catalog.plot_quicklooks(figsize=(10,10))

## Buffer by 10 meter, limit to aoi

In [ ]:
# Clip to aoi
clipped = gpd.clip(df, aoi.iloc[0].geometry)
clipped.geometry = clipped.geometry.buffer(0)

catalog.plot_coverage(scenes=clipped, aoi=aoi)

clipped.to_file(driver="GeoJSON", filename=outdir / "clipped.geojson")

## Get best non-overlapping sections. Iteratively prioritizied by cloud cover & area

In [ ]:
# Iteratively selected the next best scene and add to Dataframe. At each iteration the area criteria is recalculated, as it changes depending on the already
# selected scenes. Often times a scene which has a big area within the aoi is actually not selected, as another scene which has a similar coverage made it
# redundant.

max_incidence_angle = 30

clipped_filtered_angle = clipped.copy()
clipped_filtered_angle = clipped_filtered_angle[clipped_filtered_angle.incidenceAngle < max_incidence_angle]

full_coverage = get_best_sections_full_coverage(df=clipped_filtered_angle, order_by=["cloudCoverage"], min_size_section_sqkm=0.5)

display(full_coverage)
up42.plot_coverage(full_coverage, aoi=aoi, figsize=(7,7))

full_coverage.to_file(driver="GeoJSON", filename=outdir / "full_coverage.geojson")

# Buffer all sections by 10m (=5 Pixel 2m), so we ensure that all sections overlap later on (no segment line breaks).
# Clip again to aoi to avoid unneccesary bigger aoi and complicated geometries
full_coverage.geometry = full_coverage.geometry.apply(lambda poly: buffer_meter(poly=poly, distance=10, epsg_in=4326, 
                                                       use_centroid=False, 
                                                       lon=aoi.geometry.iloc[0].centroid.x, lat=aoi.geometry.iloc[0].centroid.x))
full_coverage = gpd.clip(full_coverage, aoi.iloc[0].geometry)
full_coverage.geometry = full_coverage.geometry.buffer(0)

full_coverage.to_file(driver="GeoJSON", filename=outdir / "full_coverage_buffered.geojson")

## Create & test workflow

In [ ]:
#selected_blocks = ["oneatlas-pleiades-aoiclipped"]
selected_blocks = ["oneatlas-pleiades-fullscene", "pansharpen"]

In [ ]:
project = up42.initialize_project()
workflow = project.create_workflow("mosaicing", use_existing=True)

blocks=up42.get_blocks(basic=True)
workflow.add_workflow_tasks([blocks[selected] for selected in selected_blocks])

In [ ]:
# Test workflow & availability of sections

test_jobs = {}
for idx, row in full_coverage.iterrows():

    test_parameters = workflow.construct_parameters(geometry=row.geometry, 
                                               geometry_operation="intersects", 
                                               scene_ids=[row["scene_id"]])
    #print(test_parameters)

    test_job = workflow.create_and_run_job(test_parameters, test_query=True)
    test_job.track_status(report_time=60)
    
    test_df = test_job.get_results_json(as_dataframe=True)
    display(test_df)
    
    test_jobs[idx] = test_df
    
print("finished")

## Download sections

In [ ]:
# Download the different sections for full coverage

jobs = {}
for idx, row in full_coverage.iterrows():

    parameters = workflow.construct_parameters(geometry=row.geometry, 
                                               geometry_operation="intersects", 
                                               scene_ids=[row["scene_id"]])
    print(parameters)

    job = workflow.create_and_run_job(parameters)
    job.track_status(report_time=60)
    
    out_filepaths = job.download_results(output_directory=outdir / "sections")
    jobs[out_filepaths[0]] = job
    
print("finished")

In [ ]:
# Get order ids

order_ids=[]
if "fullscene" in selected_blocks[0]:
    for _, job in jobs.items():
        
        data_jobtask = job.get_jobtasks()[0]
        
        order_id = data_jobtask.get_results_json()["features"][0]["orderID"]
        order_ids.append(order_id)
        
print(order_ids)

## Mosaic sections

In [ ]:
job_results = list(outdir.joinpath("sections").glob("*.tif"))
job_results

In [ ]:
src_files_to_mosaic = []
for fp in job_results:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

    out_profile = src.profile.copy() # bzw. src.meta
print(src_files_to_mosaic)


mosaic, out_transform = merge(src_files_to_mosaic)
print(mosaic.shape)



out_profile.update({
    'height': mosaic.shape[1],
    'width': mosaic.shape[2],
    'transform': out_transform,
    'blockxsize':256,
    'blockysize':256,
    'tiled' : True  # Important for definition block structure!  
})

out_path = outdir / "mosaic/mosaic_pleiades_complex.tif"
# Write raster.
with rasterio.open(out_path, 'w', **out_profile) as dst:
        for i in range(mosaic.shape[0]):
            dst.write(mosaic[i,...], i+1)

In [ ]:
# Visualize mosaic

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12,12))

show(out_path, ax=ax)
plt.show()